### Carga diaria de Analisis de Sentimientos

*  Cargar los datos
 * Cargar TRM Historica y calcular deltas
*  Calcular correlaciones spearman y pearson
 * Correlaciones con diferentes Lags hasta 15 días
* Validar otras variables

In [1]:
import pandas as pd
import tweepy
from datetime import *
import datetime as dt
import json, re
import feedparser
from time import mktime

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import mysql.connector
connection = mysql.connector.connect(host='localhost', port=3307,
                                         database='tesis',
                                         user='root',
                                         password='Cidifo03%%')

In [5]:
# Analisis de Sentimientos
##librerias ingles
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

#librerías Español
import joblib
from  classifier import SentimentClassifier
clf = SentimentClassifier()

# importaciones Tokenizacion

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
pst = PorterStemmer()
import string
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
stopWEnglish = set(stopwords.words('english'))
stopWSP = set(stopwords.words('spanish'))

from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\ProgramData\Anaconda3\envs\T\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stop

In [6]:
pd.set_option("display.max_colwidth", 100)

In [7]:
#fechaHoy=datetime.datetime.now().date()
fechaHoy=dt.datetime.now().date()

#  Funciones

In [8]:
def limpiar_terminos(texto):    
    texto=" ".join(filter(lambda x:x[0]!='#', texto.split()))
    texto=" ".join(filter(lambda x:x[0]!='@', texto.split()))
    texto=" ".join(filter(lambda x:x[0:4]!='http', texto.split()))
    return texto

In [9]:
def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """    
    texto=limpiar_terminos(texto)
    
    # Eliminamos los caracteres especiales
    texto = re.sub(r'\W', ' ', str(texto))
    texto = re.sub(r'\d', ' ', str(texto))
    # Eliminamos comas y /
    texto = re.sub(r',', ' ', str(texto))
    texto = re.sub(r'/', ' ', str(texto))
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    # remover \n
    #texto = re.sub(r'\n', '', texto)
    texto = texto.replace('\n', '')
    texto = texto.replace('º', '')    
      
    # Convertimos textos a minusculas
    texto = texto.lower()
    return texto

In [10]:
def analisisSentimientosIngles(text):
    resultado = sid.polarity_scores(text)
    return resultado["compound"]

## enEspañol
def analisisSentimientosEspanol(text):
    resultado= clf.predict(text)
    resultado= resultado*2-1
    #resultado= resultado
    return resultado


def seleccionAnalisis(fila):
    #print(fila)
    if fila["lang"] == "en" :
        return analisisSentimientosIngles(str(fila["full_text"]))
    elif fila["lang"] == "es" :
        return analisisSentimientosEspanol(str(fila["full_text"]))

#tokenizacion
def tokenizcionIngles(text):
    textPunt = text.translate(str.maketrans('','',string.punctuation))
    textPunt = textPunt.translate(str.maketrans('','','1234567890'))
    textPunt = textPunt.translate(str.maketrans('','','’'))
    textPunt = textPunt.translate(str.maketrans('','',"''"))
    textPunt = textPunt.translate(str.maketrans('','','✅'))
    textMin = word_tokenize(textPunt.lower())
    textSW = [x for x in textMin if x not in stopWEnglish]
    tokenStm = [None] * len(textSW)
    for i in range(0, len(textSW)):
        tokenStm[i]=pst.stem(textSW[i])
    return tokenStm
    #fdist = FreqDist(tokenStm)
    #return pd.DataFrame(fdist.items(), columns=['word', 'frec'])

def tokenizcionEspanol(text):
    textPunt = text.translate(str.maketrans('','',string.punctuation))
    textPunt = textPunt.translate(str.maketrans('','','1234567890'))
    textPunt = textPunt.translate(str.maketrans('','','’'))
    textPunt = textPunt.translate(str.maketrans('','',"'"))
    textPunt = textPunt.translate(str.maketrans('','','✅'))
    #quitarHasTags
    
    ##textMin = filter(lambda x:x[0]!='#', word_tokenize(textPunt.lower()))
    textMin = word_tokenize(textPunt.lower())    
    
    textSW = [x for x in textMin if x not in stopWSP]
    textSW = [x for x in textSW if x not in word_rem['Palabras'].values] 
    stemmed_text = [stemmer.stem(i) for i in textSW]
    return stemmed_text
    #fdist = FreqDist(stemmed_text)
    #return pd.DataFrame(fdist.items(), columns=['word', 'frec'])

def seleccionTokenizacion(fila,var):
    if fila["lang"] == "en" :
        return tokenizcionIngles(str(fila[var]))
    elif fila["lang"] == "es" :
        return tokenizcionEspanol(str(fila[var]))


#RSS definicion de métodos
def parse(url):
    return feedparser.parse(url)

def get_source(parsed):
    feed= parsed['feed']
    return {
        'link': feed['link'],
        'title': feed['title'],
        'subtitle': feed['subtitle']
    }

def get_articles(parsed):
        articles = []
        entries= parsed['entries']
        for entry in entries:
            articles.append({
                'id': entry['id'],
                'link': entry['link'],
                'title': entry['title'],
                'summary':entry['summary'],
                'published': entry['published_parsed']                
            })
        return articles
    
def get_articles_resultado(parsed,fuenteRSS):
        articles = []
        entries= parsed['entries']
        for entry in entries:
            #partDate =entry['published_parsed'].replace('(','')replace(')','').split(",")
            #almacenarFuente            
            # validacion si falta una dato            
            if 'summary' not in entry :
                entry['summary']=""
            if 'title' not in entry :
                entry['title']=""    
            articles.append({
                'fuenteRss':fuenteRSS,
                'idP': entry['id'],                
                'texto': entry['title']+". "+entry['summary'],
                'published': entry['published_parsed']                
            })
        return articles
    
def getFechaFromRss(fechaStruct):
    #print (fechaStruct)
    if fechaStruct is None:
        #print ("entro")
        return datetime.now()
    else:
        return datetime.fromtimestamp(mktime(fechaStruct))

In [11]:
### Funciones para modelo de tópicos
def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts,bigram_mod,trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def impresionTopicos(mod,nPalabras,nTopicos):
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    for topico in topicos:
        print(topico)    
        
def impresionTopicosLimp(mod,nPalabras,nTopicos):
    #Impresion limpia de Tópicos
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    i=0
    for topico in topicos:
        i=i+1
        textoAux = re.sub(r'\W', ' ', str(topico))
        textoAux = re.sub(r'\s+[a-zA-Z]\s+', ' ', textoAux)
        textoAux = re.sub(r'[0-9]', ' ', textoAux)
        textoAux = re.sub(r'\s+', ' ', textoAux, flags=re.I)
        print(str(i),textoAux)

## Carga de los datos

In [12]:
### Twitter
sql_select_Query = "select * from tesis.t_base_twitter"
base_twitter=pd.read_sql(sql_select_Query, connection)

In [13]:
#Rss
sql_select_Query = "select * from tesis.t_base_rss"
base_rss=pd.read_sql(sql_select_Query, connection)

### TWITER

In [14]:
base_twitter

,index,screen_name,id,id_str,name,location,Idioma,concepto,impacto,NoTweets
0,0,sergiodigra,NaN,NaN,None,None,es,None,NaN,100
1,1,Victor_Malagon,NaN,NaN,None,None,es,None,NaN,100
2,2,jscorrales,NaN,NaN,None,None,es,None,NaN,100
3,3,_minecogob,NaN,NaN,None,None,es,None,NaN,100
4,4,elEconomistaes,NaN,NaN,None,None,es,None,NaN,100
5,5,FT,NaN,NaN,None,None,en,None,NaN,100
6,6,rodriguezbraun,NaN,NaN,None,None,es,None,NaN,100
7,7,josecdiez,NaN,NaN,None,None,es,None,NaN,100
8,8,suanzes,NaN,NaN,None,None,es,None,NaN,100
9,9,wef,NaN,NaN,None,None,es,None,NaN,100


In [15]:
###TWEETER
# autenticación
# keys and tokens from the Twitter Dev Console 
consumer_key = '5BOcNnQsuIxDTBejlZ75xBqZ0'
consumer_secret = 'c6Cnx7SItN5Gda6K8hgtPo3tPyI18TU9eUh28jx3d4lDceewiG'
access_token = '927023579727884288-1jTaXF5wmUbX2NeYzUcSlmzXQMca4kU'
access_token_secret = 'Y62wUwgoWjfHGw5S0jlYH7w8ulmlRzaFcxbGjBP0wLin9'

#creación del objeto para extraer información
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#obtejo
api= tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [16]:
#Cargar Tweets
tweets = []
fechaDelTweetAnterior=fechaHoy
    
for index, row in base_twitter.iterrows():
    #for amigo in df:
    i=0
    print(str(i)+" "+row['screen_name'])
    try:
        tweetsAmigo=tweepy.Cursor(api.user_timeline, screen_name=row['screen_name'], tweet_mode="extended").items(row['NoTweets'])
        for tweet in tweetsAmigo:
            tweets.append({
                'id':tweet.id,
                'full_text':tweet.full_text,
                'created_at':tweet.created_at,
                #'lang':row['Idioma'],
                'lang':tweet.lang,
                'id_fuente':row['screen_name']
            })
    except:
        print("falla API")
    
    i=i+1 

0 sergiodigra
0 Victor_Malagon
0 jscorrales
0 _minecogob
0 elEconomistaes
0 FT
0 rodriguezbraun
0 josecdiez
0 suanzes
0 wef
0 ecofeminita
0 epeconomia
0 RevistaDinero
0 bbcmundo
0 BluRadioCo
0 elespectador
0 RevistaSemana
0 ELTIEMPO
0 elcolombiano
0 lafm
0 NoticiasRCN
0 rcnradio
0 MailOnline
0 clarincom
0 NBCNews
0 rcnmundo
0 CaracolRadio
0 NoticiasCaracol
0 NTN24
0 NASA
0 PublimetroCol
0 Adcamach
0 Adriana_Guzman
0 agaviriau
0 AlbertoBernalLE
0 alhosu
0 amaldon19
0 Anamibanez
0 BancoRepublica
0 BvcColombia
0 CamiloDeGuzman
0 carlos_g_ospino
0 castellanosgd
0 CehernandezEcon
0 Cesaretamayo
0 estebanpie
0 Fedesarrollo
0 FocoEconomico
0 Fondosdepension
0 freddykastro
0 IECarballo
0 jagallegod
0 japebu
0 JavierMejiaC
0 jleibovi
0 Jllanosala
0 JoMalagon
0 JorgeARestrepo
0 JorgeATovar
0 JoseILOPEZ
0 jrobertoacosta1
0 juanf_vargas
0 juansrobledo
0 JusTributaria
0 LeopoldoTweets
0 LuisFelipeHenao
0 LuisFerMejia
0 mahofste
0 MantillaIgnacio
0 mariovalencia01
0 MauricioCard
0 Maurotov
0 Melende

In [17]:
tweetsDF=pd.DataFrame(tweets)
pd.set_option('display.max_colwidth', -1)
tweetsDFProcesado=tweetsDF.copy()
### limpiar texto
tweetsDFProcesado["full_text_Limpio"]=tweetsDFProcesado.full_text.apply(limpiar_texto)

In [18]:
tweetsDFProcesado.lang.value_counts()

es     7194
en     1249
und    175 
ca     29  
pt     19  
fr     7   
it     5   
in     4   
ro     3   
tl     3   
no     2   
cy     2   
de     2   
et     2   
ht     2   
fi     1   
cs     1   
Name: lang, dtype: int64

In [19]:
tweetsDFProcesado['SentSQDM']=tweetsDFProcesado.full_text_Limpio.apply(sentiment.sentiment)*2-1

In [20]:
#tweetsDFProcesado['SentTesis']=tweetsDFProcesado.full_text_Limpio.apply(seleccionAnalisis)
tweetsDFProcesado['SentTesis']=tweetsDFProcesado.apply(lambda row: seleccionAnalisis(row), axis=1)

In [21]:
tweetsDFProcesado['SentSQDM']=tweetsDFProcesado['SentSQDM'].round(3)
tweetsDFProcesado['SentTesis']=tweetsDFProcesado['SentSQDM'].round(3)

In [22]:
tweetsDFProcesado.head(10)

,id,full_text,created_at,lang,id_fuente,full_text_Limpio,SentSQDM,SentTesis
0,1516183517557895181,RT @AgendaCAF: 🎉 Este año celebramos 20 años de alianza con @NoticiasGDA. @sergiodigra participó en la reunión anual del grupo para contarl…,2022-04-18 22:34:54,es,sergiodigra,rt este año celebramos años de alianza con participó en la reunión anual del grupo para contarl,-0.94,-0.94
1,1516165592503533576,"RT @Osgazu: @Afromedios @EddiXavierMarce @AgendaCAF @LuisGMurillo Gran acierto! Eddy, excelente profesional y mejor persona! Muchos éxitos!…",2022-04-18 21:23:40,es,sergiodigra,rt gran acierto eddy excelente profesional mejor persona muchos éxitos,0.99,0.99
2,1514742313318236164,RT @carlos_santiso: @Vicmunro @wef @Sfalconiglobal @Maluvillalba @IvanDuque @infopresidencia @InnpulsaCol @Ministerio_TIC Felices de haberl…,2022-04-14 23:08:04,es,sergiodigra,rt felices de haberl,-0.08,-0.08
3,1514590177796177925,"@DABarril Recomiendo la lectura de este blog. La integración de los mercados de energía eléctrica en Latinoamérica y el Caribe es esencial entre otros aspectos para la seguridad, la transición energética y la formación de precios accesibles.",2022-04-14 13:03:32,es,sergiodigra,recomiendo la lectura de este blog la integración de los mercados de energía eléctrica en latinoamérica el caribe es esencial entre otros aspectos para la seguridad la transición energética la formación de precios accesibles,-0.99,-0.99
4,1514589359017803778,"RT @DABarril: Junto a Walter Cont, escribimos en el blog Visiones acerca de los procesos de integración eléctrica en la región https://t.co…",2022-04-14 13:00:17,es,sergiodigra,rt junto walter cont escribimos en el blog visiones acerca de los procesos de integración eléctrica en la región,-0.81,-0.81
5,1514243742340980743,RT @aleboterob: Ya pueden comentar el borrador de la política pública que busca cerrar brechas socioeconómicas en los 171 municipios que co…,2022-04-13 14:06:56,es,sergiodigra,rt ya pueden comentar el borrador de la política pública que busca cerrar brechas socioeconómicas en los municipios que co,-0.98,-0.98
6,1513860936959934473,"RT @AgendaCAF: 📍🗓️🌎@Sergiogra estuvo reunido con @WHAAsstSecty, Secretario Adjunto para Asuntos del Hemisferio Occidental del @StateDept, p…",2022-04-12 12:45:48,es,sergiodigra,rt sergiogra estuvo reunido con secretario adjunto para asuntos del hemisferio occidental del,0.28,0.28
7,1513330576832479240,RT @AgendaCAF: 🌏 Ampliar nuestra presencia internacional\n🚀📈 Atraer nuevos fondos para la región \n🌿🏦 Ayudar a posicionarnos como el #BancoV…,2022-04-11 01:38:20,es,sergiodigra,rt ampliar nuestra presencia internacional atraer nuevos fondos para la región ayudar posicionarnos como el,-0.16,-0.16
8,1512929872942874628,"RT @ElkinVelasquezM: Con Canciller #Honduras @EnriqueReinaHN hablamos de vivienda, #ODS, drrollo urbano sostenible y analizaremos apoyo a a…",2022-04-09 23:06:05,es,sergiodigra,rt con canciller hablamos de vivienda drrollo urbano sostenible analizaremos apoyo a,-0.07,-0.07
9,1512644594051305476,"RT @mef_Uruguay: Directora de Política Económica Marcela Bensión, asesor Hernán Bonilla, dir de Deuda Herman Kamil y dir de Multilaterales…",2022-04-09 04:12:29,es,sergiodigra,rt directora de política económica marcela bensión asesor hernán bonilla dir de deuda herman kamil dir de multilaterales,-0.22,-0.22


# RSS

In [23]:
base_rss[:8]

,index,id,FuentesRSS,lenguaje,Fuente,Pais,tema,Concepto,funciona,concepto_estandar,impacto,inter_local,DescxErrores
0,0,1.00,http://ep00.epimg.net/rss/tags/ultimas_noticias.xml,esp,el pais,venezuela,ultimo,Últimas noticias,None,actualidad,3,None,0
1,1,2.00,http://ep00.epimg.net/rss/tags/noticias_mas_vistas.xml,esp,el pais,venezuela,más leido,Noticias más vistas,None,actualidad,6,None,0
2,2,3.00,http://ep00.epimg.net/rss/economia/portada.xml,esp,el pais,colombia,economía,Portada de Economía,None,economía,10,None,0
3,3,4.00,https://www.clarin.com/rss/lo-ultimo/,esp,el clarín,argentina,lo-ultimo,"Home, lo último",None,actualidad,6,None,0
4,4,5.00,https://www.clarin.com/rss/economia/,esp,el clarín,argentina,economia,Economía,None,economía,10,None,0
5,5,6.00,https://www.elcomercio.com/feed/,esp,el comercio,perú,None,Lo último,None,actualidad,6,None,0
6,6,7.00,http://www.elcomercio.com/rss/actualidad,esp,el comercio,perú,actualidad,Actualidad,None,actualidad,6,None,1
7,7,8.00,http://cdn.eluniversal.com/rss/avances.xml,esp,el universal,venezuela,avances,"Avances, mala",None,actualidad,6,None,1


In [24]:
#RSS
base_rss=base_rss[base_rss.DescxErrores != 1]
base_rss

#procesamiento de RSS
dicResultado=pd.DataFrame()
bandera=0

for index, row in base_rss.iterrows(): 
    fuenteRss= row["FuentesRSS"]
    idioma=row["lenguaje"]
    pais=row["Pais"]
    
    #depurar antes de procesar
    print ("procesando fuente numero: "+str(index))
    print ( "fuenteRss:"+str(fuenteRss)+" idioma:"+str(idioma)+ " topico:"+str(pais) )   
    
    try:
        auxParse=parse(fuenteRss)
        articles1=get_articles_resultado(auxParse,fuenteRss)
        #articles1    
        #convertir a dataframe
        articles2=pd.DataFrame(articles1)
        #implementacion fechas a articulos
        articles2
        #depurar fecha recibida        
    
        articles2["date"]=articles2["published"].apply(getFechaFromRss)
        articles2["lang"]=idioma
        
        articles2["texto_Limpio"]=articles2["texto"].apply(limpiar_texto)        
        
        if idioma == "eng" :
            articles2["polaridad"]=articles2["texto_Limpio"].apply(analisisSentimientosIngles) 
        elif idioma == "esp" :
            articles2["polaridad"]=articles2["texto_Limpio"].apply(analisisSentimientosEspanol)
        #analisis sentimientos 2
        articles2['polaridad_SQDM']=articles2["texto_Limpio"].apply(sentiment.sentiment)*2-1
        bandera=bandera+1
        if bandera==1:
            dicResultado=articles2
        else:
            dicResultado=dicResultado.append(articles2)
        
    except Exception as e:
        print(e)

procesando fuente numero: 0
fuenteRss:http://ep00.epimg.net/rss/tags/ultimas_noticias.xml idioma:esp topico:venezuela
procesando fuente numero: 1
fuenteRss:http://ep00.epimg.net/rss/tags/noticias_mas_vistas.xml idioma:esp topico:venezuela
procesando fuente numero: 2
fuenteRss:http://ep00.epimg.net/rss/economia/portada.xml idioma:esp topico:colombia
procesando fuente numero: 3
fuenteRss:https://www.clarin.com/rss/lo-ultimo/ idioma:esp topico:argentina
procesando fuente numero: 4
fuenteRss:https://www.clarin.com/rss/economia/ idioma:esp topico:argentina
procesando fuente numero: 5
fuenteRss:https://www.elcomercio.com/feed/ idioma:esp topico:perú
procesando fuente numero: 8
fuenteRss:http://elnacional.com.do/feed/ idioma:esp topico:venezuela
procesando fuente numero: 9
fuenteRss:http://www.eltiempo.com/rss/economia.xml idioma:esp topico:colombia
procesando fuente numero: 10
fuenteRss:http://www.eltiempo.com/rss/deportes.xml idioma:esp topico:colombia
procesando fuente numero: 11
fuenteRss

'published'
procesando fuente numero: 78
fuenteRss:http://rssfeeds.usatoday.com/UsatodaycomNation-TopStories idioma:eng topico:usa
procesando fuente numero: 79
fuenteRss:http://feeds.washingtonpost.com/rss/rss_blogpost idioma:eng topico:usa
'published'
procesando fuente numero: 80
fuenteRss:https://www.eltiempo.com/rss/mundo_latinoamerica.xml idioma:esp topico:colombia
procesando fuente numero: 81
fuenteRss:https://www.eltiempo.com/rss/mundo_eeuu-y-canada.xml idioma:esp topico:colombia
procesando fuente numero: 82
fuenteRss:http://www.eltiempo.com/rss/mundo_europa.xml idioma:esp topico:colombia
procesando fuente numero: 83
fuenteRss:http://www.eltiempo.com/rss/mundo_medio-oriente.xml idioma:esp topico:colombia
procesando fuente numero: 84
fuenteRss:http://www.eltiempo.com/rss/mundo_asia.xml idioma:esp topico:colombia
procesando fuente numero: 85
fuenteRss:https://www.eltiempo.com/rss/mundo_africa.xml idioma:esp topico:colombia
procesando fuente numero: 86
fuenteRss:http://www.eltiempo.

In [25]:
dicResultado['polaridad_SQDM']=dicResultado['polaridad_SQDM'].round(3)
dicResultado['polaridad']=dicResultado['polaridad'].round(3)

In [26]:
dicResultado.sample(10)

,fuenteRss,idP,texto,published,date,lang,texto_Limpio,polaridad,polaridad_SQDM
1,https://www.clarin.com/rss/lo-ultimo/,https://www.clarin.com/servicios/capacitacion-gratis-emprendedores-tiempo-anotarse-_0_WhtYyp3djx.html,"Capacitación gratis para emprendedores: ¿hasta cuándo hay tiempo de anotarse?. Es un programa del Gobierno de la Ciudad para aprender sobre transformación digital, gestión y finanzas. Quiénes pueden acceder.","(2022, 4, 19, 11, 47, 51, 1, 109, 0)",2022-04-19 11:47:51.000000,esp,capacitación gratis para emprendedores hasta cuándo hay tiempo de anotarse es un programa del gobierno de la ciudad para aprender sobre transformación digital gestión finanzas quiénes pueden acceder,-0.63,0.71
7,http://www.eltiempo.com/rss/deportes.xml,https://www.eltiempo.com/deportes/futbol-internacional/carlos-antonio-velez-habla-sobre-james-y-su-silencio-sobre-freddy-rincon-666000,Carlos Antonio Vélez apoya silencio de James ante muerte de Freddy Rincón. Han sido múltiples las críticas a James Rodríguez&nbsp;por no enviar un mensaje de condolencias.,"(2022, 4, 19, 2, 45, 0, 1, 109, 0)",2022-04-19 02:45:00.000000,esp,carlos antonio vélez apoya silencio de james ante muerte de freddy rincón han sido múltiples las críticas james rodríguez nbsp por no enviar un mensaje de condolencias,-0.70,-1.00
5,http://www.portafolio.co/rss/economia/gobierno,https://www.portafolio.co/economia/gobierno/el-31-del-pgn-se-asigno-durante-el-primer-trimestre-564023,El 31% del PGN se asignó durante el primer trimestre.,"(2022, 4, 13, 1, 48, 30, 2, 103, 0)",2022-04-13 01:48:30.000000,esp,el del pgn se asignó durante el primer trimestre,-0.86,-0.82
33,http://ep00.epimg.net/rss/economia/portada.xml,https://elpais.com/retina/2020/02/06/innovacion/1580971727_122808.html#?ref=rss&format=simple&link=guid,Google reducirá los anuncios intrusivos en vídeos de Chrome y YouTube a partir del 5 de agosto. Se evitarán los anuncios de más de 31 segundos antes del inicio o los que interrumpen la reproducción de vídeos de menos de ocho minutos.,"(2020, 2, 6, 6, 4, 28, 3, 37, 0)",2020-02-06 06:04:28.000000,esp,google reducirá los anuncios intrusivos en vídeos de chrome youtube partir del de agosto se evitarán los anuncios de más de segundos antes del inicio los que interrumpen la reproducción de vídeos de menos de ocho minutos,-0.96,-1.00
4,http://portafolio.co/rss/negocios/emprendimiento,https://www.portafolio.co/negocios/emprendimiento/plataforma-todo-en-uno-de-servicios-financieros-empresariales-563858,Plataforma ‘todo en uno’ de servicios financieros empresariales.,"(2022, 4, 7, 19, 55, 0, 3, 97, 0)",2022-04-07 19:55:00.000000,esp,plataforma todo en uno de servicios financieros empresariales,-0.74,0.28
36,http://ep00.epimg.net/rss/economia/portada.xml,https://elpais.com/retina/2020/02/06/talento/1580986475_313138.html#?ref=rss&format=simple&link=guid,"Lorena Gómez, las apps de una apasionada de la domótica. Lorena Gómez, responsable de HMY Retail Tech, aporta algunas claves sobre cómo integrar la tecnología en los comercios para no perder el tren de la digitalización","(2020, 2, 6, 17, 36, 7, 3, 37, 0)",2020-02-06 17:36:07.000000,esp,lorena gómez las apps de una apasionada de la domótica lorena gómez responsable de hmy retail tech aporta algunas claves sobre cómo integrar la tecnología en los comercios para no perder el tren de la digitalización,-0.09,-1.00
28,http://feeds.washingtonpost.com/rss/sports,https://www.washingtonpost.com/sports/2022/04/16/washington-commanders-ftc-attorneys-general/?utm_source=rss&utm_medium=referral&utm_campaign=wp_homepage,Allegations against Commanders could prompt action by several offices. Read more,None,2022-04-19 07:01:49.274139,eng,allegations against commanders could prompt action by several offices read more,0.00,-0.00
44,http://www.nytimes.com/services/xml/rss/nyt/World.xml,https://www.nytimes.com/2022/04/16/world/middleeast/palestinians-israel-violence.html,"Israel Steps Up Raids in Palestinian Territories in Response to Attacks. 

### Guardar en la base de datos

In [27]:
import mysql.connector
import sqlalchemy
from datetime import timedelta
import time
from suds.client import Client
import time

In [28]:

database_username = 'root'
database_password = 'Cidifo03%%'
database_ip       = '127.0.0.1:3307'
database_name     = 'tesis'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()


In [29]:
tweetsDFProcesado.drop(columns=['full_text']).to_sql(con=database_connection, name='t_tweets_new_tmp', if_exists='replace',chunksize=100)

In [30]:
dicResultado.drop(columns=['published','texto']).to_sql(con=database_connection, name='t_rss_new_tmp', if_exists='replace',chunksize=100)

## Guardar en tablas principales

In [31]:
#host="127.0.0.1:3307",
mydb = mysql.connector.connect(
  host="localhost",
  port=3307,
  user="root",
  password="Cidifo03%%",
  database="tesis"
)

In [32]:
mycursor = mydb.cursor()

In [33]:
sql_act_tweets = "delete t1 from tesis.t_tweets_new t1 inner join tesis.t_tweets_new_tmp t2 on (t1.id=t2.id and t1.created_at=t2.created_at)"
sql_act_rss = "delete t1 from tesis.t_rss_new t1 inner join tesis.t_rss_new_tmp t2 on (t1.fuenteRss=t2.fuenteRss and t1.idP=t2.idP and t1.date=t2.date)"

In [34]:
mycursor.execute(sql_act_tweets)
mydb.commit()

In [35]:
mycursor.execute(sql_act_rss)
mydb.commit()

In [36]:
tweetsDFProcesado.drop(columns=['full_text']).to_sql(con=database_connection, name='t_tweets_new', if_exists='append',chunksize=100)
dicResultado.drop(columns=['published','texto']).to_sql(con=database_connection, name='t_rss_new', if_exists='append',chunksize=100)

### Próximos pasos

* Definir si se va aguardar información de tokenización
* Definir si se van a definir tópicos generales o por fuente para segmentar los textos y buscar correlaciones.
* try cath RSS
* validar cierre conexión

# Leer y guardar variables dependientes (TRM y precios)

In [37]:
date=time.strftime('%Y-%m-%d')

In [38]:
d_start = datetime.today() - timedelta(days=15)
d_start=d_start.strftime('%Y-%m-%d')
d_start

'2022-04-04'

###  TRM


In [39]:
WSDL_URL = 'https://www.superfinanciera.gov.co/SuperfinancieraWebServiceTRM/TCRMServicesWebService/TCRMServicesWebService?WSDL'
def trm(date):
    try:
        client = Client(WSDL_URL, location=WSDL_URL, faults=True)
        trm =  client.service.queryTCRM(date)
    except Exception as e:
        return str(e)

    return trm

print (trm(date))
resultado_trm=trm(date)

(tcrmResponse){
   id = 1399551
   unit = "COP"
   validityFrom = 2022-04-19 00:00:00-05:00
   validityTo = 2022-04-19 00:00:00-05:00
   value = 3731.31
   success = True
 }


In [40]:
resultado_trm

In [41]:
## Crear diccionario para guardar resultado
resultado_trm_dic={}

In [42]:
for i,j in resultado_trm:
    print(i,j)
    resultado_trm_dic[i]=j

id 1399551
unit COP
validityFrom 2022-04-19 00:00:00-05:00
validityTo 2022-04-19 00:00:00-05:00
value 3731.31
success True


In [43]:
resultado_trm_dic

{'id': 1399551,
 'unit': 'COP',
 'validityFrom': datetime.datetime(2022, 4, 19, 0, 0, tzinfo=<suds.sax.date.FixedOffsetTimezone object at 0x00000201261585B0>),
 'validityTo': datetime.datetime(2022, 4, 19, 0, 0, tzinfo=<suds.sax.date.FixedOffsetTimezone object at 0x0000020126158130>),
 'value': 3731.31,
 'success': True}

In [44]:
pd.DataFrame.from_dict(resultado_trm_dic, orient='index')

,0
id,1399551
unit,COP
validityFrom,2022-04-19 00:00:00-05:00
validityTo,2022-04-19 00:00:00-05:00
value,3731.31
success,True


In [45]:
resultado_trm_dic=pd.DataFrame.from_dict(resultado_trm_dic, orient='index').T
resultado_trm_dic

,id,unit,validityFrom,validityTo,value,success
0,1399551,COP,2022-04-19 00:00:00-05:00,2022-04-19 00:00:00-05:00,3731.31,True


In [46]:
date

'2022-04-19'

In [47]:
try:
    resultado_trm_dic.to_sql(con=database_connection, name='t_trm_new', if_exists='append',chunksize=100)
except:
    print('ya habia sido guardada')

ya habia sido guardada


## Precios
https://ichi.pro/es/predecir-los-precios-del-petroleo-con-aprendizaje-automatico-y-python-55256159639482
https://ichi.pro/es/python-he-probado-la-api-de-quandl-y-como-obtener-datos-economicos-y-inmobiliarios-en-una-linea-de-codigo-11078875261371

In [1]:
import quandl

In [3]:
d_start="2021-12-30"
date="2022-04-19"

In [4]:
# Setting our API key
quandl.ApiConfig.api_key = 'X6mZSY79yeySfpCVJaKg'

# Importing our data
data_petroleo = quandl.get("FRED/DCOILBRENTEU", start_date=d_start, end_date=date)

https://fred.stlouisfed.org/series/DCOILBRENTEU

In [5]:
data_petroleo.tail(10)

,Value
Date,
2022-02-15,98.43
2022-02-16,97.44
2022-02-17,95.28
2022-02-18,96.18
2022-02-21,98.95
2022-02-22,98.73
2022-02-23,99.29
2022-02-24,101.29
2022-02-25,98.56


In [51]:
date

'2022-04-19'

In [6]:
data_oro = quandl.get("LBMA/GOLD", start_date=d_start, end_date=date)

In [7]:
data_oro[['USD (PM)']]

,USD (PM)
Date,
2021-12-30,1805.85
2021-12-31,NaN
2022-01-04,1811.40
2022-01-05,1826.25
2022-01-06,1789.35
...,...
2022-04-08,1941.40
2022-04-11,1951.55
2022-04-12,1960.85


In [19]:
data_oro[data_oro['USD (PM)']=='NAN']

,USD (AM),USD (PM),GBP (AM),GBP (PM),EURO (AM),EURO (PM),valor_final
Date,,,,,,,


In [15]:
data_oro.isnull().sum()

USD (AM)       0
USD (PM)       1
GBP (AM)       0
GBP (PM)       1
EURO (AM)      0
EURO (PM)      1
valor_final    0
dtype: int64

In [10]:
import numpy as np

In [11]:
data_oro['valor_final']=np.where(data_oro['USD (PM)'].isnull(),data_oro['USD (AM)'],data_oro['USD (PM)'])

In [16]:
df_precios=data_petroleo.merge(data_oro[['valor_final']],left_index=True,right_index=True,how='outer')

In [54]:
#df_precios=data_petroleo.merge(data_oro[['USD (PM)']],left_index=True,right_index=True,how='outer')

In [17]:
df_precios.columns=['petroleo','oro']

In [22]:
df_precios.fillna(method='bfill',inplace=True)

In [23]:
df_precios.isnull().sum()

petroleo    33
oro          0
dtype: int64

In [28]:
df_precios.tail(40)

,petroleo,oro
Date,,
2022-02-18,96.18,1893.60
2022-02-21,98.95,1894.45
2022-02-22,98.73,1900.10
2022-02-23,99.29,1904.70
2022-02-24,101.29,1936.30
2022-02-25,98.56,1884.80
2022-02-28,103.08,1909.85
2022-03-01,NaN,1922.00
2022-03-02,NaN,1928.50


In [ ]:
df_precios

In [56]:
df_precios.to_sql(con=database_connection, name='t_precios_tmp', if_exists='replace')

In [57]:
sql_act_precios = "delete t1 from tesis.t_precios t1 inner join tesis.t_precios_tmp t2 on (t1.Date=t2.Date)"

In [58]:
mycursor.execute(sql_act_precios)
mydb.commit()

In [59]:
df_precios.to_sql(con=database_connection, name='t_precios', if_exists='append')

## Cerrar conexiones

In [60]:
### Cerrar nuevamente al final del script
connection.close()
database_connection.close()
mydb.close()